In [ ]:
from pystac_client import Client  
from collections import defaultdict    
import json
import geopandas
import geoviews as gv
from cartopy import crs
import matplotlib.pyplot as plt
from datetime import datetime
import os
import requests
from lxml import etree
import boto3
import numpy as np
import xarray as xr
import rasterio as rio
from rasterio.session import AWSSession
from rasterio.plot import show
import rioxarray
import geoviews as gv
import hvplot.xarray
import holoviews as hv
from tqdm import tqdm
from pprint import pprint
import time
gv.extension('bokeh', 'matplotlib')

## Load images and visualize

In [ ]:
print(path_dicts['Green'])

In [ ]:
def time_index_from_filenames(file_links):
    return [datetime.strptime(f.split('.')[-5], '%Y%jT%H%M%S') for f in file_links]

In [ ]:
time = xr.Variable('time', time_index_from_filenames(path_dicts['Green']))
chunks=dict(band=1, x=512, y=512)
hls_ts_da_GRN = xr.concat([rioxarray.open_rasterio(f, chunks=chunks).squeeze('band', drop=True) for f in path_dicts['Green']], dim=time)
#hls_ts_da_NIR = xr.concat([rioxarray.open_rasterio(f, chunks=chunks).squeeze('band', drop=True) for f in path_dicts['NIR']], dim=time)
#hls_ts_da_GRN = hls_ts_da_GRN.rio.reproject("epsg:4326")
#hls_ts_da_NIR = hls_ts_da_NIR.rio.reproject("epsg:4326")

In [ ]:
hls_ts_da_data_GRN = hls_ts_da_GRN.load()
hls_ts_da_data_NIR = hls_ts_da_NIR.load()
hls_ts_da_data_GRN = hls_ts_da_data_GRN.rio.clip([roi])
hls_ts_da_data_NIR = hls_ts_da_data_NIR.rio.clip([roi])

In [ ]:
hls_ts_da_data_GRN.hvplot.image(x='x', y='y', rasterize=True, width=600, height=400, colorbar=True, cmap='gray').opts(clim=(0,2000))

In [ ]:
## Caclulate Normalized Difference Water Index (NDWI) and Classify Innundated Areas

In [ ]:
GRN = hls_ts_da_data_GRN 
NIR = hls_ts_da_data_NIR
NDWI = (GRN-NIR)/(GRN+NIR)
NDWI.hvplot.image(x='x', y='y', rasterize=True, width=600, height=400, colorbar=True, cmap='coolwarm').opts(clim=(-0.5,0.5))

In [ ]:
water = NDWI>0
water.hvplot.image(x='x', y='y', rasterize=True, width=600, height=400, colorbar=True, cmap='PuOr').opts(clim=(0,1))

## Caclulate surface area of reservoir and plot time series

In [ ]:
if water.variable.max() == True:
    water_real = water*30*30
water_area = water_real.sum(axis=(1,2))

%matplotlib inline

fig, ax = plt.subplots()
(water_area[:]/1000000).plot(ax=ax, linewidth=2, linestyle = '-',  marker='o')
ax.set_title("Surface area of waterbody in km2")
ax.set_ylabel('Area [km^2]')